In [1]:
#imports
from joblib import load
import pickle
from sklearn.ensemble import RandomForestClassifier
from google.cloud import storage
import pandas as pd

In [2]:
# create storage client to download data
storage_client = storage.Client()
public_bucket = storage_client.bucket('sba_data_uoftcloudrj')
blob = public_bucket.blob('new_data.csv')
blob.download_to_filename('new_data.csv')
blob = public_bucket.blob('model.pkl')
blob.download_to_filename('model.pkl')

In [3]:
df = pd.read_csv('new_data.csv',index_col='LoanNr_ChkDgt')

In [4]:
df.head()

,Zip,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,SBA_Appv
LoanNr_ChkDgt,,,,,,,,,,,,,,
2086325003,67219,44,2007,84,4,1.0,0,4,0,1,0,0,15000.0,12750.0
1808175004,33172,53,2006,49,12,1.0,0,12,0,1,0,0,150000.0,50000.0
5628895009,79925,42,2013,9,4,1.0,0,4,0,1,0,0,32000.0,16000.0
3844064006,4090,72,2000,12,21,1.0,0,21,1,2,1,0,130000.0,50000.0
3405335009,5301,44,2009,60,0,2.0,6,6,0,2,0,0,41000.0,20500.0


In [5]:
#load model
model = pickle.load(open('model.pkl','rb'))
result = model.predict(df)
print(result)

[0 1 0 ... 0 0 0]


In [6]:
#len(result)

In [7]:
df['prediction'] = result

In [8]:
df['prediction'] = df['prediction'].map({0:'paid in full',1:"charged off"})

In [9]:
df.to_csv('predicted_data.csv')

In [10]:
storage_client = storage.Client()
bucket = storage_client.bucket('sba_data_uoftcloudrj')
blob = bucket.blob('predicted_data.csv')
blob.upload_from_filename('predicted_data.csv')